## Setting up Couchbase Cluster
- Create a cluster by going to the [Web Console](http://localhost:8091/) and selecting `Setup New Cluster`.

![image](./img/CouchbaseStartup.png)

- Create a Cluster name, username & password for the database user like `Training`, `Administrator` & `Password`

![image](./img/SetupCluster.png)

- Configure the cluster. You need not change the defaults in the configuration unless you are running low on resources.

### Note
In case you are running out of resources to run the server, you can disable the Analytics Service, reduce the memory requirements for the other services until it runs on your machine by choosing the Configure the cluster option.

![image](./img/ConfigureServices.png)



## Import Travel Sample Bucket 
For the purpose of this training, we will work with one of the sample buckets provided by Couchbase, `travel-sample`. 

You  can import it by going to the Settings Tab & clicking on `Sample Buckets`.

![image](img/ImportData.png)


## Tour of Couchbase Web Console
### Dashboard

The Dashboard gives a real-time sense of the cluster's activity and performance.

At this point, the graphs shouldn't show much activity, since we haven't sent any requests into the database yet.

![image](./img/Dashboard.png)

### Servers 

On the Servers screen, you can see that we have only a single server (127.0.0.1) in the cluster in this playground installation.

The Servers screen is where you can add and remove servers from the cluster. It also specifies the services running on the cluster and their resource usage.

![image](./img/Servers.png)



## Buckets, Scopes & Collections
### Buckets
Couchbase Server keeps data in Buckets.

Buckets store data persistently, as well as in memory. They allow data to be automatically replicated for high availability, using the Database Change Protocol (DCP); and dynamically scaled across multiple clusters, by means of Cross Datacenter Replication (XDCR).

![image](./img/Buckets.png)

### Scopes & Collections
They allow documents to be categorized and organized, within a bucket. 

A bucket holds containers called scopes, which in turn holds containers called collections, which in turn holds JSON documents.

A collection is a data container, defined on Couchbase Server, within a bucket. Up to 1000 collections can be created per cluster. Item-names must be unique within their collection. Items can optionally be assigned to different collections according to content-type. For example, within a bucket that contains travel information, documents that relate specifically to airports might be assigned to an airports collection, while documents that relate to hotels might be assigned to a hotels collection, and so on. Applications can be assigned per-collection access-rights; allowing each application to access only those collections it requires.

A scope is a mechanism for the grouping of multiple collections. Up to 1000 scopes can be created per cluster. Collection-names must be unique within their scope. Collections might be assigned to different scopes according to content-type, or to deployment-phase (ie, test versus production). Applications can be assigned per-scope access-rights; allowing each application to access only those scopes it requires.

![image](./img/Scopes_and_Collections.png)

### Documents
Documents are data points stored in Couchbase. They consist of a key by which the item is referenced; and an associated value, which must be either binary or a JSON document. The values can be upto 20MB in size.

In the Documents Workbench screen, you can find and modify your JSON documents.

The Keyspace control lets you examine the documents in a specific bucket/scope/collection.

![image](./img/Documents.png)

## Mapping of Couchbase Concepts with Relational Database Concepts
The concepts in Couchbase can be mapped to the Relational Database concepts that you might already be familiar with.

![SQL-To-Couchbase Mapping](./img/SQL_To_Couchbase.png)

## Query Workbench
Query Workbench is an interface that supports the querying of data by means of the SQL++ (formerly N1QL) query language. It utilizes the Query Service which depends on the Index & Data Services.

The Query Workbench has a Query Editor, where you can edit and execute SQL++ queries (i.e., SQL for JSON).

![image](./img/Query_Workbench.png)

## Indexes

Indexes lists all the indexes defined in Couchbase. They help with improving the performance of queries.

It is important to note that in the absence of Indexes, you cannot perform any SQL++ Queries on the data stored in Couchbase. For the `travel-sample` data, we have the indexes created when we import the data. However if you are working with your data, you need to ensure that you have the indexes required to access the data. We will cover this in Lab 03.

![image](./img/Indexing.png)

## Python SDK
In this course, we will be accessing Couchbase using the Python SDK.

The Couchbase Python SDK allows Python applications to access a Couchbase cluster. The Python SDK (v4 onwards) uses the high-performance C++ library libcouchbase to handle communicating to the cluster over Couchbase’s binary protocols.

### Installation
`python3 -m pip install couchbase`



## Running the Code Samples
- You can run each of the cells one by one (Press Ctrl+Run or Click on Run button in the menu) and observe the outputs. 
- If you face any problems with the samples, it is most likely due to the overwriting of some variables in the Notebook. You can fix it by running the cell that contains the variable that is missing. Or run the notebook from the beginning.
- Feel free to explore by changing the options and/or queries. 

### Configuring the Couchbase Cluster Information for Examples

The configuration is stored in an environment file, `.env` in this folder. 

Note that you might have to check for hidden files to see this file on Unix environments.

This file can be used to update the connection settings.
* DB_HOST: Set to `couchbase://couchbase` by default for connecting to the Couchbase cluster in the docker environment via Docker Compose. If you are running Couchbase locally on your machine via docker or installation, you can change the connection string to `couchbase://localhost`.
* DB_USER: Set to `Administrator` by default. If it is different for your cluster, please update the file.
* DB_PASSWORD: Set to `Password` by default. If it is different for your cluster, please update the file.


In [ ]:
# Read the Database information from .env file
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env file.

In [ ]:
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
print(f"Environment Settings \n{DB_HOST=} \n{DB_USER=} \n{DB_PASSWORD=}")

### Connecting to Couchbase Cluster
- Connection String: `couchbase://couchbase` would connect to the Couchbase instance.
- PasswordAuthenticator: It specifies the username & password used to access the Cluster.

#### Note
If you are running Couchbase locally on your machine via docker or installation, you can change the connection string to `couchbase://localhost` via the configuration file `.env`

In [ ]:
from couchbase.auth import PasswordAuthenticator

# needed to support SQL++ (N1QL) query
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions, QueryOptions

# get a reference to our cluster
cluster = Cluster(DB_HOST, ClusterOptions(PasswordAuthenticator(DB_USER, DB_PASSWORD)))

### Accessing a Bucket within Cluster

In [ ]:
# get a reference to our bucket
cb = cluster.bucket("travel-sample")

### Accessing the Collection within a Bucket

In [ ]:
cb_coll = cb.scope("inventory").collection("airline")

### Reading Data using Queries
The queries are run using the Cluster object

In [ ]:
n1ql_query = "SELECT * from `travel-sample`.inventory.airline LIMIT 10"
query_results = cluster.query(n1ql_query)

In [ ]:
# process the results from the query
for result in query_results:
    print(result)

### Data Models

In [ ]:
# Pretty Printing
import pprint

pp = pprint.PrettyPrinter(indent=4, depth=6)

In [ ]:
# Airlines
n1ql_query = "SELECT * from `travel-sample`.inventory.airline LIMIT 1"
query_results = cluster.query(n1ql_query)
result = list(query_results)
pp.pprint(result[0])

In [ ]:
# Airports
n1ql_query = "SELECT * from `travel-sample`.inventory.airport LIMIT 1"
query_results = cluster.query(n1ql_query)
result = list(query_results)
pp.pprint(result[0])

In [ ]:
# Hotels
n1ql_query = "SELECT * from `travel-sample`.inventory.hotel LIMIT 1"
query_results = cluster.query(n1ql_query)
result = list(query_results)
pp.pprint(result[0])

## Exercise 1.1
Examine the data models for 
- landmarks
- routes

In [ ]:
# Landmarks


In [ ]:
# Routes


## References
- [Overview of Couchbase](https://docs.couchbase.com/server/current/learn/architecture-overview.html)
- [Introduction to the Couchbase Python SDK](https://docs.couchbase.com/python-sdk/current/hello-world/overview.html)